## Metrics Search Requests
This notebook explains how to use the API to retrieve citation data and other metrics for a given bibcode(s). Examples here will be shown using Python with the use of the [requests](http://docs.python-requests.org/en/master/) library, though the same work could be done using the [unofficial Python ADS library](https://ads.readthedocs.io/en/latest/) or curl commands on the command line (see the "API documentation - UNIX shell" folder in this same repository).

The **base_url** for metrics queries is `https://api.adsabs.harvard.edu/v1/metrics`. The given endpoint in each section should be appended to the end of the base URL to make the full URL to be used in the request. 

In all examples below, `token` should be replaced with [your own API token](https://ui.adsabs.harvard.edu/user/settings/token). If you haven't worked with our API before, it's recommended that you read the [README](https://github.com/adsabs/adsabs-dev-api/blob/master/README.md) before beginning.

In [1]:
# import the requests package and set your token in a variable for later use
import requests
import json # the json package will be used for passing the POST payload - see the section below on sending a request

token="your-token-here"

### Sending a metrics request
All metrics requests use the HTTP POST method. POST requests require a payload to be specified using the `data` keyword. For metrics, this payload will be JSON formatted, with the key `bibcodes` and a value set to a comma-separated list of the bibcodes to retrieve metrics for. We use the `json` library here to convert between Python dictionaries and JSON strings. For metrics requests, the payload accepts the following keys:

* `bibcodes`: **required**; set the value to a list of the bibcodes to retrieve metrics for
* `types`: optional; set the value to a list of the types of metrics to retrieve. If `types` is not set, all metrics will be retrieved. The following values are allowed for the `types` list:

  * `basic': publication and usage stats (all papers, and just refereed papers)
  * `citations': citation stats
  * indicators': indicators, like the h-index, g-index, m-index etc etc
  * 'histograms': publication, citation, reads and downloads histograms
  * 'timeseries': time series for a set of indicators
  
* `histograms`: optional; if `histograms` is specified under `types`, this keyword can be set to specify the histograms to return. The following values are allowed (more information is given below):

  * `publications`
  * `reads`
  * `downloads`
  * `citations`

### Response from a metrics request
The service returns JSON with attributes determined by the contents of the `types` list (no such list means that everything gets returned). It always contains an attribute `skipped bibcodes`, which is a list of bibcodes for which no metrics data could be found. The following JSON attributes are returned by each type:

  * 'basic': 'basic stats' and 'basic stats refereed'
  * 'citations': 'citation stats' and 'citation stats refereed'
  * 'indicators': 'indicators' and 'indicators refereed'
  * 'histograms': 'histograms'
  * 'timeseries': 'time series'

Each of these attributes contains specific data elements. The following sections shows what each type contains (each bullet point is an attribute name, or JSON key).

#### 'basic stats' and 'basic stats refereed'
  * number of papers
  * normalized paper count
  * median number of reads
  * average number of reads
  * total number of reads
  * normalized number of reads
  * median number of downloads
  * average number of downloads
  * total number of downloads
  * normalized number of downloads
  
#### 'citation stats' and 'citation stats refereed'
  * number of citing papers
  * median number of citations
  * average number of citations
  * total number of citations
  * normalized number of citations
  * median number of refereed citations
  * average number of refereed citations
  * total number of refereed citations
  * normalized number of refereed citations
  * number of self-citations

#### 'indicators'
  * i10
  * i100
  * h
  * m
  * g
  * read10
  * tori

#### 'histograms'
For all histograms, the actual histogram values are stored as follows:
```
    {
	    "histogram name": {year: value, year: value, ...}
	}
```
where values can be either integers or floats, depending on the type of histograms. The histogram name is an attribute of the histogram type. For example, the `publications` type has 4 separate histograms: `all publications`, `refereed publications`, `all publications normalized` and `refereed publications normalized`, which are organized as follows in the JSON output:
```
    {
	    "histograms":  {
			"publications": {
				"all publications": {},
				"refereed publications": {},
				"all publications normalized": {},
				"refereed publications normalized": {},
			
			}
		}
	}
```
The following lists all histogram types and the names of the actual histograms stored in them:

  * 'publications': 'all publications', 'refereed publications', 'all publications normalized', 'refereed publications normalized'
  * 'reads': 'all reads', 'refereed reads', 'all reads normalized', 'refereed reads normalized'
  * 'downloads': 'all downloads', 'refereed downloads', 'all downloads normalized', 'refereed downloads normalized'
  * 'citations': 'refereed to refereed', 'nonrefereed to refereed', 'refereed to nonrefereed', 'nonrefereed to nonrefereed', 'refereed to refereed normalized', 'nonrefereed to refereed normalized', 'refereed to nonrefereed normalized', 'nonrefereed to nonrefereed normalized'

Note that 'refereed reads' means: 'number of reads for the refereed publications in the set'.

#### 'timeseries'
The `timeseries` attribute contains time series data for: `h`, `g`, `i10`, `i100`, `read10` and `tori`. The output is structured as
```
    {
		"time series": {
			"h": { values },
			"g": { values },
			"i10": { values },
			"i100": { values },
			"read10": { values },
			"tori": { values },
		
		}
	}
```
where the 'values' are organized in the same way as for the histograms: 
```
    {
	    "indicator name": {year: value, year: value, ...}
	}
```

### Examples

In [2]:
# get all metrics for a single bibcode
payload = {"bibcodes": ["2003ApJS..148..175S"]}
results = requests.post("https://api.adsabs.harvard.edu/v1/metrics", \
                       headers={'Authorization': 'Bearer ' + token, 
                                "Content-type": "application/json"}, \
                       data=json.dumps(payload))
results.json()

{'skipped bibcodes': [],
 'basic stats': {'number of papers': 1,
  'normalized paper count': 0.058823529411764705,
  'total number of reads': 22882,
  'average number of reads': 22882.0,
  'median number of reads': 22882.0,
  'recent number of reads': 110,
  'total number of downloads': 9122,
  'average number of downloads': 9122.0,
  'median number of downloads': 9122.0,
  'recent number of downloads': 43},
 'basic stats refereed': {'number of papers': 1,
  'normalized paper count': 0.058823529411764705,
  'total number of reads': 22882,
  'average number of reads': 22882.0,
  'median number of reads': 22882.0,
  'recent number of reads': 110,
  'total number of downloads': 9122,
  'average number of downloads': 9122.0,
  'median number of downloads': 9122.0,
  'recent number of downloads': 43},
 'citation stats': {'number of citing papers': 9724,
  'number of self-citations': 0,
  'self-citations': [],
  'total number of citations': 9724,
  'average number of citations': 9724.0,
  'm

In [3]:
# get basic metrics for multiple bibcodes
payload = {"bibcodes": ["2003ApJS..148..175S", "2007ApJS..170..377S"],
          "types": ["basic"]}
results = requests.post("https://api.adsabs.harvard.edu/v1/metrics", \
                       headers={'Authorization': 'Bearer ' + token, 
                                "Content-type": "application/json"}, \
                       data=json.dumps(payload))
results.json()

{'skipped bibcodes': [],
 'basic stats': {'number of papers': 2,
  'normalized paper count': 0.10427807486631016,
  'total number of reads': 41887,
  'average number of reads': 20943.5,
  'median number of reads': 20943.5,
  'recent number of reads': 173,
  'total number of downloads': 16779,
  'average number of downloads': 8389.5,
  'median number of downloads': 8389.5,
  'recent number of downloads': 66},
 'basic stats refereed': {'number of papers': 2,
  'normalized paper count': 0.10427807486631016,
  'total number of reads': 41887,
  'average number of reads': 20943.5,
  'median number of reads': 20943.5,
  'recent number of reads': 173,
  'total number of downloads': 16779,
  'average number of downloads': 8389.5,
  'median number of downloads': 8389.5,
  'recent number of downloads': 66}}

In [4]:
# get data for a single histogram for multiple bibcodes
payload = {"bibcodes": ["2003ApJS..148..175S", "2007ApJS..170..377S"],
          "types": ["histograms"],
          "histograms": ["citations"]}
results = requests.post("https://api.adsabs.harvard.edu/v1/metrics", \
                       headers={'Authorization': 'Bearer ' + token, 
                                "Content-type": "application/json"}, \
                       data=json.dumps(payload))
results.json()

{'skipped bibcodes': [],
 'histograms': {'citations': {'refereed to refereed': {'1998': 0,
    '1999': 0,
    '2000': 0,
    '2001': 2,
    '2002': 0,
    '2003': 383,
    '2004': 979,
    '2005': 997,
    '2006': 1440,
    '2007': 1722,
    '2008': 1670,
    '2009': 1106,
    '2010': 924,
    '2011': 773,
    '2012': 651,
    '2013': 591,
    '2014': 483,
    '2015': 425,
    '2016': 341,
    '2017': 326,
    '2018': 306,
    '2019': 310,
    '2020': 284,
    '2021': 310,
    '2022': 282,
    '2023': 46},
   'refereed to nonrefereed': {'1998': 0,
    '1999': 0,
    '2000': 0,
    '2001': 0,
    '2002': 0,
    '2003': 0,
    '2004': 0,
    '2005': 0,
    '2006': 0,
    '2007': 0,
    '2008': 0,
    '2009': 0,
    '2010': 0,
    '2011': 0,
    '2012': 0,
    '2013': 0,
    '2014': 0,
    '2015': 0,
    '2016': 0,
    '2017': 0,
    '2018': 0,
    '2019': 0,
    '2020': 0,
    '2021': 0,
    '2022': 0,
    '2023': 0},
   'nonrefereed to refereed': {'1998': 1,
    '1999': 0,
    '2000': 0